In [2]:
# import warnings
# warnings.simplefilter(action='ignore')

# import pandas as pd
# import numpy as np
# import datetime

# #抓取百度数据
# from bs4 import BeautifulSoup
# from selenium import webdriver
# import numpy as np
# import pandas as pd
# import re
# import os
# import time

# from datetime import datetime,timedelta
# thedate=datetime.strftime(datetime.today()-timedelta(days=0),"%Y-%m-%d %H-%M")

# browser = webdriver.Chrome()

# url='https://xljk.bechangedt.com:9290/fever#/'

# browser.get(url)
# time.sleep(3)

# with open(r'./HZ_hospital/%s.html' % thedate,'w',encoding='utf-8') as f:
#     f.write(browser.page_source)

# browser.close()

In [2]:
# import warnings
# warnings.simplefilter(action='ignore')

# import pandas as pd
# import numpy as np
# import datetime



# from bs4 import BeautifulSoup
# import lxml
# import os

# do=pd.DataFrame(data=['1 拥挤','2 忙碌','3 畅通','4 其他'],columns=['status'])
# for f in os.listdir(r'./HZ_hospital/'):
#     soup=BeautifulSoup(open(r'./HZ_hospital/%s' % f,'r',encoding='utf-8').read())

#     trs=soup.find_all('div',{"class":'FoldableCard--cardItem--3J0S-sY'})

#     trs

#     def get_result(tr):
#         name=tr.find('div',{'class':'FoldableCard--title--1GLuzCG'}).text
#         tag=tr.find('div',{'class':'FoldableCard--tagWrap--2bIDCZF'}).text
#         add=tr.find('div',{'class':'FoldableCard--address--B2_k5fk'}).text
#         return name,tag,add

#     d=[get_result(tr) for tr in trs]

#     df=pd.DataFrame(data=d,columns=['hosp','tag','addr'])
#     c1=df['tag'].str.contains('可查看实时队列')
#     c2=df['addr'].str.contains('杭州')
#     df=df[c1&c2]

#     def getstatus(tag):
#         if '拥挤' in tag:
#             return '1 拥挤'
#         if '忙碌' in tag:
#             return '2 忙碌'
#         if '畅通' in tag:
#             return '3 畅通'
#         else:
#             return '4 其他'

#     df['status']=df['tag'].apply(getstatus)
#     d1=df.groupby(by='status').count().reset_index()[['status','hosp']]
#     d1=d1.rename(columns={'hosp':f.replace('.html','')})
#     do=do.merge(d1,how='left',on='status')
# do.to_clipboard(index=False)
# # df=df.merge(city_popu,left_on='city',right_on='城市')
# # c1=df['status'].str.contains('其他')
# # df[~c1].reset_index(drop=True)
# do

,status,2022-12-22 21-20,2022-12-22 22-10,2022-12-22 22-59,2022-12-23 10-24,2022-12-23 11-36,2022-12-23 14-03,2022-12-23 15-29,2022-12-23 16-58,2022-12-23 19-36,...,2022-12-26 17-02,2022-12-26 17-20,2022-12-26 19-49,2022-12-26 21-33,2022-12-27 12-11,2022-12-27 15-45,2022-12-27 17-41,2022-12-27 19-30,2022-12-27 22-21,2022-12-27 23-49
0,1 拥挤,14,13,11,8,10,10,11,10,11,...,10,10,11,NaN,7,10,8,5,3,3
1,2 忙碌,4,5,6,6,5,2,3,7,6,...,5,5,5,NaN,3,3,5,4,6,5
2,3 畅通,6,6,7,7,5,8,7,5,5,...,7,8,7,NaN,13,10,11,15,14,15
3,4 其他,6,6,6,9,10,10,9,8,8,...,10,9,9,32.0,9,10,9,9,10,10


In [1]:
import os
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime

dfo=pd.DataFrame()

for f in os.listdir('./DataSource/ZhejiangHospital/')[1:]:
    print(f)
    d=pd.read_csv('./DataSource/ZhejiangHospital\%s' % f)
    dfo=pd.concat([dfo,d],axis=0)
# dfo.tail(10)

zhejiang.fever.2022-12-27.csv
zhejiang.fever.2022-12-28.csv
zhejiang.fever.2022-12-29.csv
zhejiang.fever.2022-12-30.csv
zhejiang.fever.2022-12-31.csv
zhejiang.fever.2023-01-01.csv
zhejiang.fever.2023-01-02.csv
zhejiang.fever.2023-01-03.csv
zhejiang.fever.2023-01-04.csv


In [23]:
df=dfo.copy()
c1=df['seeQueue']==True
c2=df['orgAddr'].str.contains('杭州市')
df=df[c1&c2]
df['date']=pd.to_datetime(df['抓取时间']).dt.date
df=df[['orgName','date','hour','currentStatus']].groupby(by=['orgName','date','hour']).sum().reset_index()



def getstatus(tag):
    tag=str(tag)
    re='4 其他'
    if '畅通' in tag:
        re= '3 畅通'
    if '忙碌' in tag:
        re= '2 忙碌'
    if '拥挤' in tag:
        re= '1 拥挤'
        
    return re
df['status']=df['currentStatus'].apply(getstatus)
# df=df.groupby(by=['date','hour','status']).count().reset_index()
df['datehour']=df['date'].astype("string")+'-'+df['hour'].astype("string")
d=pd.pivot_table(df,index=['date','hour'],columns='status',values='orgName',aggfunc='count').reset_index()

In [26]:
d=d.fillna(0)
d['N']=d['1 拥挤']+d['2 忙碌']+d['3 畅通']+d['4 其他']
d['拥挤率']=(d['1 拥挤']+d['2 忙碌'])/d['N']
pd.pivot_table(d,columns='date',index='hour',values='拥挤率',aggfunc='sum').to_clipboard()